In [4]:
import sys  
sys.path.insert(0, '../')
from utils import *

In [5]:
X_train = np.load('./0/data/X_train.npy')
X_test = np.load('./0/data/X_test.npy')
X_val = np.load('./0/data/X_val.npy')
y_train = np.load('./0/data/y_train.npy')
y_test = np.load('./0/data/y_test.npy')
y_val = np.load('./0/data/y_val.npy')

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Tuning RF

In [29]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 45, num = 3)]
# Minimum number of samples required to split a node
min_samples_split = [5, 10]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
              'random_state': [0]}

print(random_grid)

{'n_estimators': [20, 65, 110, 155, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [1, 23, 45], 'min_samples_split': [5, 10], 'random_state': [0]}


In [30]:
%%time
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=23, max_features=sqrt, min_samples_split=10, n_estimators=200, random_state=0; total time= 2.3min
[CV] END max_depth=23, max_features=sqrt, min_samples_split=10, n_estimators=200, random_state=0; total time= 1.3min
[CV] END max_depth=23, max_features=sqrt, min_samples_split=10, n_estimators=200, random_state=0; total time= 1.2min
[CV] END max_depth=23, max_features=sqrt, min_samples_split=10, n_estimators=200, random_state=0; total time= 1.2min
[CV] END max_depth=23, max_features=sqrt, min_samples_split=10, n_estimators=200, random_state=0; total time= 1.3min
[CV] END max_depth=1, max_features=sqrt, min_samples_split=10, n_estimators=110, random_state=0; total time=   3.4s
[CV] END max_depth=1, max_features=sqrt, min_samples_split=10, n_estimators=110, random_state=0; total time=   3.4s
[CV] END max_depth=1, max_features=sqrt, min_samples_split=10, n_estimators=110, random_state=0; total time=   3.6s
[CV] E

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [1, 23, 45],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_split': [5, 10],
                                        'n_estimators': [20, 65, 110, 155, 200],
                                        'random_state': [0]},
                   verbose=2)

In [31]:
rf_random.best_estimator_ , rf_random.best_score_

(RandomForestClassifier(max_depth=23, max_features='sqrt', min_samples_split=10,
                        n_estimators=200, random_state=0),
 0.8752380952380951)

In [32]:
type(rf_random.best_estimator_)

sklearn.ensemble._forest.RandomForestClassifier

In [33]:
RF = rf_random.best_estimator_

In [34]:
RF.fit(X_train,y_train)
y_pred_val = RF.predict(X_val)
y_pred_test = RF.predict(X_test)

print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

val accuracy: 0.8816428571428572
test accuracy: 0.8741428571428571


In [35]:
if not os.path.exists('./model_params/'):
    os.makedirs('./model_params')

# with open('./model_params/GB_params_tuning.pkl', 'wb') as f:
#         pickle.dump(GBDT_random.best_estimator_, f)
with open('./model_params/RF_params_tuning.pkl', 'wb') as f:
        pickle.dump(rf_random.best_estimator_, f)

# GB tuning

In [36]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 4)]
learning_rate = [x for x in np.linspace(start = 0.05, stop = 0.2, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(start = 2, stop = 10, num = 4)]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'learning_rate' : [0.05, 0.1, 0.15, 0.2],
               'max_features': max_features,
               'max_depth': max_depth,
              'random_state': [0]}

pprint(random_grid)

{'learning_rate': [0.05, 0.1, 0.15, 0.2],
 'max_depth': [2, 4, 7, 10],
 'max_features': ['auto', 'sqrt'],
 'n_estimators': [10, 23, 36, 50],
 'random_state': [0]}


In [37]:
%%time
GBDT_random = RandomizedSearchCV(estimator = GradientBoostingClassifier(), param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2)
# Fit the random search model
GBDT_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END learning_rate=0.1, max_depth=4, max_features=sqrt, n_estimators=23, random_state=0; total time=  36.1s
[CV] END learning_rate=0.1, max_depth=4, max_features=sqrt, n_estimators=23, random_state=0; total time=  36.8s
[CV] END learning_rate=0.1, max_depth=4, max_features=sqrt, n_estimators=23, random_state=0; total time=  36.3s
[CV] END learning_rate=0.1, max_depth=4, max_features=sqrt, n_estimators=23, random_state=0; total time=  35.7s
[CV] END learning_rate=0.1, max_depth=4, max_features=sqrt, n_estimators=23, random_state=0; total time=  35.6s
[CV] END learning_rate=0.1, max_depth=10, max_features=auto, n_estimators=10, random_state=0; total time=15.6min
[CV] END learning_rate=0.1, max_depth=10, max_features=auto, n_estimators=10, random_state=0; total time=15.6min
[CV] END learning_rate=0.1, max_depth=10, max_features=auto, n_estimators=10, random_state=0; total time=15.6min
[CV] END learning_rate=0.1, max_depth=10

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(),
                   param_distributions={'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [2, 4, 7, 10],
                                        'max_features': ['auto', 'sqrt'],
                                        'n_estimators': [10, 23, 36, 50],
                                        'random_state': [0]},
                   verbose=2)

In [38]:
GBDT_random.best_estimator_ , GBDT_random.best_score_

(GradientBoostingClassifier(learning_rate=0.2, max_depth=7, max_features='sqrt',
                            n_estimators=50, random_state=0),
 0.8823571428571428)

In [39]:
GB = rf_random.best_estimator_
GB.fit(X_train,y_train)
y_pred_val = GB.predict(X_val)
y_pred_test = GB.predict(X_test)

print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

val accuracy: 0.8816428571428572
test accuracy: 0.8741428571428571


In [49]:
if not os.path.exists('./model_params/'):
    os.makedirs('./model_params')

with open('./model_params/GB_params_tuning.pkl', 'wb') as f:
        pickle.dump(GBDT_random.best_estimator_, f)
with open('./model_params/RF_params_tuning.pkl', 'wb') as f:
        pickle.dump(rf_random.best_estimator_, f)

In [4]:
from tensorflow.keras.optimizers import SGD
pixels  = int(sqrt(X_train.shape[1]))
classes = len(np.unique(y_test))

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
# compile model
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.save(f'./model_params/CNN_params_tuning')

INFO:tensorflow:Assets written to: ./model_params/CNN_params_tuning\assets


In [5]:
num_labels = classes
pixels = int(sqrt(X_train.shape[1]))
# Reshaping to format which CNN expects (batch, height, width, channels)
trainX_cnn = X_train.reshape(X_train.shape[0], pixels, pixels, 1).astype('float32')
valX_cnn = X_val.reshape(X_val.shape[0], pixels, pixels, 1).astype('float32')
testX_cnn= X_test.reshape(X_test.shape[0], pixels, pixels, 1).astype('float32')

# Normalize images from 0-255 to 0-1
trainX_cnn /= 255
valX_cnn /= 255

train_y_cnn = utils.to_categorical(y_train, num_labels)
val_y_cnn = utils.to_categorical(y_val, num_labels)
test_y_cnn = utils.to_categorical(y_test, num_labels)

In [46]:
model.fit(trainX_cnn, train_y_cnn, epochs=10, batch_size=32, validation_data=(valX_cnn, val_y_cnn))

Epoch 1/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.4571 - accuracy: 0.8366 - val_loss: 0.3338 - val_accuracy: 0.8814
Epoch 2/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.3048 - accuracy: 0.8889 - val_loss: 0.3142 - val_accuracy: 0.8851
Epoch 3/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.2565 - accuracy: 0.9055 - val_loss: 0.2632 - val_accuracy: 0.9036
Epoch 4/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.2269 - accuracy: 0.9162 - val_loss: 0.2660 - val_accuracy: 0.9044
Epoch 5/10
1313/1313 [==============================] - 21s 16ms/step - loss: 0.2017 - accuracy: 0.9252 - val_loss: 0.2809 - val_accuracy: 0.9012
Epoch 6/10
1313/1313 [==============================] - 21s 16ms/step - loss: 0.1794 - accuracy: 0.9333 - val_loss: 0.2541 - val_accuracy: 0.9111
Epoch 7/10
1313/1313 [==============================] - 20s 15ms/step - loss: 0.1596 - accuracy: 0.9411 - val_loss: 0.2590 - val

In [47]:
y_pred_val=np.argmax(model.predict(valX_cnn),axis=1)
y_pred_test=np.argmax(model.predict(testX_cnn),axis=1)

In [48]:
print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

val accuracy: 0.905
test accuracy: 0.8714285714285714


# 1-NN

In [7]:
OneNN_model = KNeighborsClassifier(n_neighbors=1).fit(X_train, y_train)

In [8]:
y_pred_val = OneNN_model.predict(X_val)
y_pred_test = OneNN_model.predict(X_test)
print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

val accuracy: 0.8485
test accuracy: 0.8412142857142857


In [9]:
if not os.path.exists('./model_params/'):
    os.makedirs('./model_params')
with open('./model_params/OneNN_params_tuning.pkl', 'wb') as f:
        pickle.dump(OneNN_model, f)